In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc
from scipy.stats import ttest_ind
import statistics
from matplotlib.patches import Rectangle


In [ ]:
#importing the PRSice-2 output .best file (converetd into .txt in terminal). Here I add the sep to seperate the column names with a comma) 
df_POAG = pd.read_table('/mnt/shared_folders/eResearch_glaucoma_project/Sirithi/16_02_new_case_control_and_cov_file/PRS_output/trial04.best', sep=r'\s+',header=0, encoding='ascii',engine='python')
df_Craig = pd.read_table('/mnt/shared_folders/eResearch_glaucoma_project/Sirithi/16_02_new_case_control_and_cov_file/PRS_output/trial05.best', sep=r'\s+',header=0, encoding='ascii',engine='python')


In [ ]:
#importing the case control file 
df_cc = pd.read_table('/mnt/shared_folders/eResearch_glaucoma_project/Sirithi/16_02_new_case_control_and_cov_file/Treated_diagnosed_case_control.txt', sep=r'\s+',header=0, encoding='ascii',engine='python')

In [ ]:
4# Thought I should only use data thats used in the regression model
df_POAG_filtered = df_POAG[df_POAG['In_Regression'] == 'Yes']
df_Craig_filtered = df_Craig[df_Craig['In_Regression'] == 'Yes']

In [ ]:
#drop NA
df_POAG_filtered = df_POAG_filtered.dropna(subset=['PRS'])
df_Craig_filtered = df_Craig_filtered.dropna(subset=['PRS'])

In [ ]:
# convert PRS column to numeric
df_POAG_filtered['PRS'] = pd.to_numeric(df_POAG_filtered['PRS'], errors='coerce')
df_Craig_filtered['PRS'] = pd.to_numeric(df_Craig_filtered['PRS'], errors='coerce')

In [ ]:
print(df_Craig_filtered)

In [ ]:
# Merge the two PRS files based on common columns (FID and IID)
merged_prs = pd.merge(df_Craig_filtered, df_POAG_filtered, on=['FID', 'IID'])


In [ ]:
print(merged_prs)

In [ ]:
# Add the PRS from both files
merged_prs['Combined_PRS'] = merged_prs['PRS_x'] + merged_prs['PRS_y']


In [ ]:
# Merge the merged PRS with the case-control file based on common columns (FID and IID)
merged_data = pd.merge(merged_prs, df_cc, on=['FID', 'IID'])


In [ ]:
# Separate the data into cases and controls based on 'Phenotypes'
cases = merged_data[merged_data['Phenotypes'] == 1]['Combined_PRS']
controls = merged_data[merged_data['Phenotypes'] == 0]['Combined_PRS']


In [ ]:
# standardize the PRS data
scaler = StandardScaler()
merged_data['PRS_standardized'] = scaler.fit_transform(merged_data['Combined_PRS'].values.reshape(-1, 1))

# plot the standardized normal distribution curves with different colors and make it prety
plt.figure(figsize=(10, 6))
sns.histplot(data=merged_data, x='PRS_standardized', bins=30, hue='Phenotypes', kde=True, stat='density', common_norm=False)

plt.title('Standardized Normal Distribution of PRS for Cases and Controls for UKBiobank Data')
plt.xlabel('Standardized PRS')
plt.ylabel('Density')
plt.legend(title='Group', labels=[ 'Cases','Controls'])
plt.show()

In [ ]:
# calculating the PRS deciles
merged_data['PRS_decile']= pd.qcut(merged_data['PRS_standardized'], q=10, labels=False)


In [ ]:
# counting hte number of indv (cases vs controls)
decile_counts = merged_data.groupby(['PRS_decile', 'Phenotypes']).size().unstack(fill_value=0)


In [ ]:
# calculatin the propitons 
decile_proportions = decile_counts.div(decile_counts.sum(axis=1), axis=0)


In [ ]:
plt.figure(figsize=(12, 6))
barplot = sns.barplot(x=decile_proportions.index, y=decile_proportions[1], color='darkorange', label='Cases')
sns.barplot(x=decile_proportions.index, y=decile_proportions[0], color='skyblue', label='Controls', bottom=decile_proportions[1])


for p, case_count in zip(barplot.patches, decile_counts[1]):
    height = p.get_height()
    ymin, ymax = plt.ylim()
    position = ymax - 0.99 * (ymax - ymin) 
    barplot.text(p.get_x() + p.get_width() / 2,
                 position,
                 f'{case_count}',
                 ha='center')

# Annotate each bar with separate counts for controls
for p, control_count in zip(barplot.patches, decile_counts[0]):
    height = p.get_height() + decile_proportions[1]
    ymin, ymax = plt.ylim()
    position = ymax - 0.1 * (ymax - ymin) 
    barplot.text(p.get_x() + p.get_width() / 2,
                 position,
                 f'{control_count}',
                 ha='center')
    
plt.title('Proportion of Cases and Controls in PRS Deciles with Counts')
plt.xlabel('PRS Decile')
plt.ylabel('Proportion')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

plt.figure(figsize=(18, 10))


In [ ]:
import numpy as np
import scipy.stats as stats

# Calculate mean and standard deviation for cases and controls
mean_cases = np.mean(merged_data.loc[merged_data['Phenotypes'] == 1, 'PRS_standardized'])
mean_controls = np.mean(merged_data.loc[merged_data['Phenotypes'] == 0, 'PRS_standardized'])

std_cases = np.std(merged_data.loc[merged_data['Phenotypes'] == 1, 'PRS_standardized'])
std_controls = np.std(merged_data.loc[merged_data['Phenotypes'] == 0, 'PRS_standardized'])

# Calculate sample size for cases and controls
n_cases = len(merged_data.loc[merged_data['Phenotypes'] == 1, 'PRS_standardized'])
n_controls = len(merged_data.loc[merged_data['Phenotypes'] == 0, 'PRS_standardized'])

# Calculate pooled standard error of the mean difference
se_mean_diff = np.sqrt((std_cases**2 / n_cases) + (std_controls**2 / n_controls))

# Calculate t-statistic
t_statistic = (mean_cases - mean_controls) / se_mean_diff

# Calculate degrees of freedom
df = n_cases + n_controls - 2

# Calculate two-sided p-value
p_value = stats.t.sf(np.abs(t_statistic), df) * 2

# Calculate 95% confidence interval for the mean difference
diff_ci = stats.t.interval(0.95, df, loc=(mean_cases - mean_controls), scale=se_mean_diff)

# Format the output
mean_diff = mean_cases - mean_controls
output = f"Mean PRS for Cases: {mean_cases:.4f} ± {std_cases:.4f}\n" \
         f"Mean PRS for Controls: {mean_controls:.4f} ± {std_controls:.4f}\n" \
         f"Mean difference (Cases - Controls): {mean_diff:.4f}\n" \
         f"95% CI for the mean difference: {diff_ci}\n" \
         f"P-value: {p_value:.4f}"

print(output)


In [ ]:
# Extract counts for the first (lowest) and tenth (highest) deciles
cases_1st_decile = decile_counts.loc[decile_counts.index.min(), 1]
controls_1st_decile = decile_counts.loc[decile_counts.index.min(), 0]
cases_10th_decile = decile_counts.loc[decile_counts.index.max(), 1]
controls_10th_decile = decile_counts.loc[decile_counts.index.max(), 0]

# Calculate odds for the first and tenth deciles
odds_1st_decile = cases_1st_decile / controls_1st_decile
odds_10th_decile = cases_10th_decile / controls_10th_decile

# Calculate odds ratio
odds_ratio = odds_10th_decile / odds_1st_decile

# Calculate the standard error (SE) of the log odds ratio
log_odds_1st_decile = np.log(odds_1st_decile)
log_odds_10th_decile = np.log(odds_10th_decile)

SE_log_OR = np.sqrt(1 / cases_1st_decile + 1 / controls_1st_decile + 1 / cases_10th_decile + 1 / controls_10th_decile)

# Calculate the 95% CI for the odds ratio
CI_lower = np.exp(np.log(odds_ratio) - 1.96 * SE_log_OR)
CI_upper = np.exp(np.log(odds_ratio) + 1.96 * SE_log_OR)

print(f'Odds Ratio between the highest and lowest risk deciles: {odds_ratio:.2f}')
print(f'95% CI: [{CI_lower:.2f}, {CI_upper:.2f}]')

In [ ]:
## OKay so ROC and AUC - 

X = merged_data[['PRS_standardized']]
y = merged_data['Phenotypes']

# Standardize the PRS data
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)

In [ ]:

#ROC curve
fpr, tpr, thresholds = roc_curve(y, X)

#AUC score
roc_auc = auc(fpr, tpr)

# Plotting the ROC curve
plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()



In [ ]:
# T-test to see the difference b/w the means
#using ttest_ind because its not paired- cases and controls are independant 

t_statistic, p_value = ttest_ind(cases, controls, equal_var=False)
print(f'T-stat: {t_statistic:.4f}')
print(f'P-value: {p_value:.10g}')
p_value

#confused with the p-value? Thats too perfect 

In [ ]:
print (statistics.mean(cases))
print (statistics.mean(controls))

In [ ]:
import os
# Create a new figure for the combined plots
plt.figure(figsize=(21, 6))  # Increase the width to add more space between the plots

# Plot 1: Standardized Normal Distribution of PRS
plt.subplot(1, 3, 1)
scaler = StandardScaler()
merged_data['PRS_standardized'] = scaler.fit_transform(merged_data['Combined_PRS'].values.reshape(-1, 1))
sns.histplot(data=merged_data, x='PRS_standardized', bins=30, hue='Phenotypes', kde=True, stat='density', common_norm=False)
plt.xlabel('Standardized PRS', fontsize=16)
plt.ylabel('Density', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=12)
plt.legend( labels=['Cases', 'Controls'], fontsize=12, frameon=False)
plt.text(-0.1, 1, '(A)', color='black', fontsize=16, va='center', ha='left', transform=plt.gca().transAxes, fontweight='bold')


# Plot 2: Proportion of Cases and Controls in PRS Deciles
plt.subplot(1, 3, 2)
barplot = sns.barplot(x=decile_proportions.index, y=decile_proportions[1], color='darkorange', label='Cases')
sns.barplot(x=decile_proportions.index, y=decile_proportions[0], color='skyblue', label='Controls', bottom=decile_proportions[1])

for i, p in enumerate(barplot.patches[:10]):
    height = p.get_height()
    ymin, ymax = plt.ylim()
    position = ymax - 0.99 * (ymax - ymin)
    barplot.text(p.get_x() + p.get_width() / 2, position, f'{decile_counts[1][i]}', ha='center')

for i, p in enumerate(barplot.patches[10:]):
    height = p.get_height() + decile_proportions[1][i]
    ymin, ymax = plt.ylim()
    position = ymax - 0.1 * (ymax - ymin)
    barplot.text(p.get_x() + p.get_width() / 2, position, f'{decile_counts[0][i]}', ha='center')

plt.xticks(np.arange(0, 10), ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
plt.xlabel('PRS Decile', fontsize=16)
plt.ylabel('Proportion', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=12)
plt.text(-0.1, 1, '(B)', color='black', fontsize=16, va='center', ha='left', transform=plt.gca().transAxes, fontweight='bold')

# Custom legend for Cases and Controls
plt.legend([], frameon=False)
plt.text(1.02, 0.1, 'Cases', color='darkorange', fontsize=14, va='center', ha='left', transform=plt.gca().transAxes, rotation=90, fontweight='bold')
plt.text(1.02, 0.8, 'Controls', color='skyblue', fontsize=14, va='center', ha='left', transform=plt.gca().transAxes, rotation=90, fontweight='bold')

# Plot 3: ROC Curve
plt.subplot(1, 3, 3)
fpr, tpr, thresholds = roc_curve(y, X_standardized)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)', fontsize=16)
plt.ylabel('True Positive Rate (TPR)', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=12)
plt.legend(loc='lower right', frameon=False, fontsize=14)  # Increase font size for the legend
plt.text(-0.1, 1, '(C)', color='black', fontsize=16, va='center', ha='left', transform=plt.gca().transAxes, fontweight='bold')

# Display the combined figure
plt.tight_layout()
# Save the figure
folder_path = '/mnt/shared_folders/eResearch_glaucoma_project/Sirithi/Graph_Images/facet_plots/'
file_path = os.path.join(folder_path, 'POAG_Crag_PRS_addition_allindv.pdf')

# Ensure the folder exists and save the figure
os.makedirs(folder_path, exist_ok=True)
plt.savefig(file_path)

plt.close()



### Lets look at TD 

In [ ]:
#importing the PRSice-2 output .best file (converetd into .txt in terminal). Here I add the sep to seperate the column names with a comma) 
df_POAG = pd.read_table('/mnt/shared_folders/eResearch_glaucoma_project/Sirithi/Glaucoma_definition/PRS_outcomes/Treated_diagnosed.best', sep=r'\s+',header=0, encoding='ascii',engine='python')
df_Craig = pd.read_table('/mnt/shared_folders/eResearch_glaucoma_project/Sirithi/Glaucoma_definition/PRS_outcomes/Craig_Treated_diagnosed.best', sep=r'\s+',header=0, encoding='ascii',engine='python')


In [ ]:
#importing the case control file 
df_cc = pd.read_table('/mnt/shared_folders/eResearch_glaucoma_project/Sirithi/16_02_new_case_control_and_cov_file/Treated_diagnosed_case_control.txt', sep=r'\s+',header=0, encoding='ascii',engine='python')

In [ ]:
4# Thought I should only use data thats used in the regression model
df_POAG_filtered = df_POAG[df_POAG['In_Regression'] == 'Yes']
df_Craig_filtered = df_Craig[df_Craig['In_Regression'] == 'Yes']

In [ ]:
#drop NA
df_POAG_filtered = df_POAG_filtered.dropna(subset=['PRS'])
df_Craig_filtered = df_Craig_filtered.dropna(subset=['PRS'])

In [ ]:
# convert PRS column to numeric
df_POAG_filtered['PRS'] = pd.to_numeric(df_POAG_filtered['PRS'], errors='coerce')
df_Craig_filtered['PRS'] = pd.to_numeric(df_Craig_filtered['PRS'], errors='coerce')

In [ ]:
print(df_Craig_filtered)

In [ ]:
# Merge the two PRS files based on common columns (FID and IID)
merged_prs = pd.merge(df_Craig_filtered, df_POAG_filtered, on=['FID', 'IID'])


In [ ]:
print(merged_prs)

In [ ]:
# Add the PRS from both files
merged_prs['Combined_PRS'] = merged_prs['PRS_x'] + merged_prs['PRS_y']


In [ ]:
# Merge the merged PRS with the case-control file based on common columns (FID and IID)
merged_data = pd.merge(merged_prs, df_cc, on=['FID', 'IID'])


In [ ]:
# Separate the data into cases and controls based on 'Phenotypes'
cases = merged_data[merged_data['Phenotypes'] == 1]['Combined_PRS']
controls = merged_data[merged_data['Phenotypes'] == 0]['Combined_PRS']


In [ ]:
# standardize the PRS data
scaler = StandardScaler()
merged_data['PRS_standardized'] = scaler.fit_transform(merged_data['Combined_PRS'].values.reshape(-1, 1))

# plot the standardized normal distribution curves with different colors and make it prety
plt.figure(figsize=(10, 6))
sns.histplot(data=merged_data, x='PRS_standardized', bins=30, hue='Phenotypes', kde=True, stat='density', common_norm=False)

plt.title('Standardized Normal Distribution of PRS for Cases and Controls for UKBiobank Data')
plt.xlabel('Standardized PRS')
plt.ylabel('Density')
plt.legend(title='Group', labels=[ 'Cases','Controls'])
plt.show()

In [ ]:
# Calculate summary statistics for cases and controls
summary_stats = merged_data.groupby('Phenotypes')['PRS_standardized'].agg(['mean', 'median', 'std', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)])
summary_stats.columns = ['Mean', 'Median', 'Standard Deviation', '25th Percentile (Q1)', '75th Percentile (Q3)']
summary_stats['IQR'] = summary_stats['75th Percentile (Q3)'] - summary_stats['25th Percentile (Q1)']

# Display the summary statistics
print(summary_stats)

In [ ]:
# calculating the PRS deciles
merged_data['PRS_decile']= pd.qcut(merged_data['PRS_standardized'], q=10, labels=False)


In [ ]:
# counting hte number of indv (cases vs controls)
decile_counts = merged_data.groupby(['PRS_decile', 'Phenotypes']).size().unstack(fill_value=0)


In [ ]:
# calculatin the propitons 
decile_proportions = decile_counts.div(decile_counts.sum(axis=1), axis=0)


In [ ]:
plt.figure(figsize=(12, 6))
barplot = sns.barplot(x=decile_proportions.index, y=decile_proportions[1], color='darkorange', label='Cases')
sns.barplot(x=decile_proportions.index, y=decile_proportions[0], color='skyblue', label='Controls', bottom=decile_proportions[1])


for p, case_count in zip(barplot.patches, decile_counts[1]):
    height = p.get_height()
    ymin, ymax = plt.ylim()
    position = ymax - 0.99 * (ymax - ymin) 
    barplot.text(p.get_x() + p.get_width() / 2,
                 position,
                 f'{case_count}',
                 ha='center')

# Annotate each bar with separate counts for controls
for p, control_count in zip(barplot.patches, decile_counts[0]):
    height = p.get_height() + decile_proportions[1]
    ymin, ymax = plt.ylim()
    position = ymax - 0.1 * (ymax - ymin) 
    barplot.text(p.get_x() + p.get_width() / 2,
                 position,
                 f'{control_count}',
                 ha='center')
    
plt.title('Proportion of Cases and Controls in PRS Deciles with Counts')
plt.xlabel('PRS Decile')
plt.ylabel('Proportion')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

plt.figure(figsize=(18, 10))


In [ ]:
## OKay so ROC and AUC - 

X = merged_data[['PRS_standardized']]
y = merged_data['Phenotypes']

# Standardize the PRS data
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)

In [ ]:

#ROC curve
fpr, tpr, thresholds = roc_curve(y, X)

#AUC score
roc_auc = auc(fpr, tpr)

# Plotting the ROC curve
plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()



In [ ]:
# T-test to see the difference b/w the means
#using ttest_ind because its not paired- cases and controls are independant 

t_statistic, p_value = ttest_ind(cases, controls, equal_var=False)
print(f'T-stat: {t_statistic:.4f}')
print(f'P-value: {p_value:.10g}')
p_value

#confused with the p-value? Thats too perfect 

In [ ]:

# Create a new figure for the combined plots
plt.figure(figsize=(21, 6))  # Increase the width to add more space between the plots

# Plot 1: Standardized Normal Distribution of PRS
plt.subplot(1, 3, 1)
scaler = StandardScaler()
merged_data['PRS_standardized'] = scaler.fit_transform(merged_data['Combined_PRS'].values.reshape(-1, 1))
sns.histplot(data=merged_data, x='PRS_standardized', bins=30, hue='Phenotypes', kde=True, stat='density', common_norm=False)
plt.xlabel('Standardized PRS', fontsize=16)
plt.ylabel('Density', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=12)
plt.legend( labels=['Cases', 'Controls'], fontsize=12, frameon=False)
plt.text(-0.1, 1, '(A)', color='black', fontsize=16, va='center', ha='left', transform=plt.gca().transAxes, fontweight='bold')


# Plot 2: Proportion of Cases and Controls in PRS Deciles
plt.subplot(1, 3, 2)
barplot = sns.barplot(x=decile_proportions.index, y=decile_proportions[1], color='darkorange', label='Cases')
sns.barplot(x=decile_proportions.index, y=decile_proportions[0], color='skyblue', label='Controls', bottom=decile_proportions[1])

for i, p in enumerate(barplot.patches[:10]):
    height = p.get_height()
    ymin, ymax = plt.ylim()
    position = ymax - 0.99 * (ymax - ymin)
    barplot.text(p.get_x() + p.get_width() / 2, position, f'{decile_counts[1][i]}', ha='center')

for i, p in enumerate(barplot.patches[10:]):
    height = p.get_height() + decile_proportions[1][i]
    ymin, ymax = plt.ylim()
    position = ymax - 0.1 * (ymax - ymin)
    barplot.text(p.get_x() + p.get_width() / 2, position, f'{decile_counts[0][i]}', ha='center')

plt.xticks(np.arange(0, 10), ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
plt.xlabel('PRS Decile', fontsize=16)
plt.ylabel('Proportion', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=12)
plt.text(-0.1, 1, '(B)', color='black', fontsize=16, va='center', ha='left', transform=plt.gca().transAxes, fontweight='bold')

# Custom legend for Cases and Controls
plt.legend([], frameon=False)
plt.text(1.02, 0.1, 'Cases', color='darkorange', fontsize=14, va='center', ha='left', transform=plt.gca().transAxes, rotation=90, fontweight='bold')
plt.text(1.02, 0.8, 'Controls', color='skyblue', fontsize=14, va='center', ha='left', transform=plt.gca().transAxes, rotation=90, fontweight='bold')

# Plot 3: ROC Curve
plt.subplot(1, 3, 3)
fpr, tpr, thresholds = roc_curve(y, X_standardized)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)', fontsize=16)
plt.ylabel('True Positive Rate (TPR)', fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=12)
plt.legend(loc='lower right', frameon=False, fontsize=14)  # Increase font size for the legend
plt.text(-0.1, 1, '(C)', color='black', fontsize=16, va='center', ha='left', transform=plt.gca().transAxes, fontweight='bold')

# Display the combined figure
plt.tight_layout()
# Save the figure
folder_path = '/mnt/shared_folders/eResearch_glaucoma_project/Sirithi/Graph_Images/facet_plots/'
file_path = os.path.join(folder_path, 'POAG_Crag_PRS_addition_TD.pdf')

# Ensure the folder exists and save the figure
os.makedirs(folder_path, exist_ok=True)
plt.savefig(file_path)

plt.close()

In [ ]:

# Save the figure
folder_path = '/mnt/shared_folders/eResearch_glaucoma_project/Sirithi/Graph_Images/facet_plots/'
file_path = os.path.join(folder_path, 'POAG_Crag_PRS_addition_TD.pdf')

# Ensure the folder exists and save the figure
os.makedirs(folder_path, exist_ok=True)
plt.savefig(file_path)

plt.close()